# Porous Electrode FiPy model

Troubleshooting of model presented in [usnistgov/fipy#788](https://github.com/usnistgov/fipy/issues/788)

In [1]:
%matplotlib inline

$ \frac{\partial c_1}{\partial t} = -\nabla\cdot N_1 - \nabla\cdot i/F $

$ \frac{\partial c_2}{\partial t} = -\nabla\cdot N_2 + \nabla\cdot i/F $

$ \sum{z_i \nabla\cdot N_i} = \nabla\cdot i/F $

where

$ N_i = -D \nabla\cdot c_i - Dz_ic_i \nabla\cdot \phi_2 $

and

$ \nabla\cdot i = a i0/por(c_1/c_1^0 e^{z_1 ct F/(RT)(\phi_1 - \phi_2 - \phi_0)} - c_2/c_2^0 e^{z_2 ct F/(RT)(\phi_1 - \phi_2 - \phi_0)}) $

$a$, $i0$, $F$,$D$,$\phi_1$,$z_1$,$ct$,$R$,$T$, $c_1^0$, and $c_2^0$ are constants
$\phi_2$, $c_1$, and $c_2$ are time-dependent, time-evolving variables to solve for. In the problem, $z_2$ is 0 which simplifies the flux and exponent terms for $c_2$.

In implementing the flux terms in my code given by N, the chain rule is used to better couple the system and try to stay implicit:

$\nabla (c \nabla \phi) \equiv \nabla c \cdot \nabla \phi + c \nabla^2 \phi$

For simplicity, start with 0 flux boundary conditions on the bounds of x on [0,L].
The initial conditions are:

$ c_1(x,0) = 0.1 $

$ c_2(x,0) = 0. $

$ \phi_2 (x,0) = \phi_0 $

In [ ]:
import numpy as np
import scipy
from fipy import *
from fipy import Variable, FaceVariable, CellVariable, Grid1D, TransientTerm, DiffusionTerm, Viewer
from fipy import ImplicitSourceTerm, ExplicitDiffusionTerm
from fipy.tools import numerix
from builtins import range

phi_0 = 0.0 # arbitrarily set to 0
phi_app = 0.2 # Applied voltage 
c1_0 = 0.1 # M  
c2_0 = 0.1 # M
z1 = -1.0
z2 = 0.
phi_1 = phi_0 + phi_app/2 
L = .25 # cm of electrode thickness 
por = 0.4 # porosoity 
D = 5*10**-5 # diffusion coefficient, cm^2/s
a = 360. #cm^2 per cm^3, specific interfacial area for GFD2.5
ct,R,F,T = 0.5, 8.3143, 96487., 293.
ne, s = 1, 1
i0 = 0.005 # from Exchange Current Densities 

# New code
nx = 10000
dx = L/nx
mesh = Grid1D(nx=nx,dx=dx)
x = mesh.cellCenters[0]
dt = .01
#dt = 0.5*dx**2/(2*D)
steps = 100
t = dt * steps
time = Variable(0.)

phi2 = CellVariable(name="phi2",mesh=mesh,hasOld=True)
c1 =  CellVariable(name="c1",mesh=mesh,hasOld=True)
c2 =  CellVariable(name="c2",mesh=mesh,hasOld=True)
phi2.setValue(phi_0)
c1.setValue(c1_0)
c2.setValue(c2_0)

# No flux at x=0, dc2/dx = 0 automatically
# Flux for c1 has concentration & potential gradient
# Set up Robin BC;
# unsure about this set up; folllowed this website:
# https://www.ctcms.nist.gov/fipy/documentation/
#USAGE.html#applying-robin-boundary-conditions

#Make point to apply Robin BC, where x=0
mask = (x=0.)
# Create values to put into equation for phi2
phi_a = FaceVariable(mesh=mesh,rank=1)
phi_a.setValue(-2.,where=mask)
phi_b = FaceVariable(mesh=mesh,rank=0)
phi_b.setValue(1.,where=mask)
phi_g =  FaceVariable(mesh=mesh,rank=0)
phi_g.setValue(-2.0*(phi_1+phi_0),where=mask)

# Repeat for c1 whose concentration gradient 
# cancels the migration gradient
c1_a = FaceVariable(mesh=mesh,rank=1)
c1_a.setValue(-1.,where=mask)
c1_b = FaceVariable(mesh=mesh,rank=0)
c1_b.setValue(1.,where=mask)

c1.faceGrad.constrain(0,mesh.facesRight)
phi2.faceGrad.constrain(0,mesh.facesRight)

# No gradients at x = L
c1.faceGrad.constrain(0,mesh.facesRight)
c2.faceGrad.constrain(0,mesh.facesRight)
phi2.faceGrad.constrain(0,mesh.facesRight)

# Constants in equations
beta = ct*F/R/T # For exponential
mig = D*F/R/T # in front of migration term
bv = a*i0/F/por # in front of Faradaic reaction

# Equation for first species
eq1 = (TransientTerm(var=c1) == DiffusionTerm(coeff=D,var=c1) 
    # Migration effect (is this OK??)
    + DiffusionTerm(coeff=c1*z1*mig,var=phi2)
    # subtract Faradaic term
    - bv*c1/c1_0*numerix.exp(z1*beta*(phi_1-phi2-phi_0))
    + bv*c2/c2_0  
      # incorporate Robin BC at x =0
    + PowerLawConvectionTerm(coeff=c1_a)
    + DiffusionTerm(coeff=b) )

# Equation for second species
eq2 = (TransientTerm(var=c2) == DiffusionTerm(coeff=D,var=c2)
    + bv*c1/c1_0*numerix.exp(z1*beta*(phi_1-phi2-phi_0)) 
    - bv*c2/c2_0  )

# Equation relating flux to current for electrolyte potential
eq3 = (DiffusionTerm(coeff=c1*mig,var=phi2) +  mig*c1.grad.dot(phi2.grad) 
    # Add diffusion
    # + D*z1*c1.faceGrad.divergence <- don't use this?
    # + DiffusionTerm(coeff=z1*D*c1,var=None) <- don't use!
    + DiffusionTerm(coeff=z1*D,var=c1)
    # Add Faradaic contributions, 
    # can't use ImplicitSourceTerm because it's nonlinear in phi
    + bv*(c1/c1_0*numerix.exp(z1*beta*(phi_1-phi2-phi_0)) 
    - c2/c2_0) 
      # Add Robin BCs
    + PowerLawConvectionTerm(coeff=phi_a)
    + DiffusionTerm(coeff=phi_b) + (g*mask*mesh.faceNormals).divergence )


# Need to do Newton steps
deltaC1 = CellVariable(mesh=mesh,name="delta C1",hasOld=True)
deltaC2 = CellVariable(mesh=mesh,name="delta C2",hasOld=True)
deltaPhi2 = CellVariable(mesh=mesh,name="delta Phi2",hasOld=True)

# Add new Newton Equations; need to do derivatives and fill in
newton_eqC1 = (TransientTerm(var=deltaC1)
            == DiffusionTerm(coeff=D*c1.grad,var=deltaC1) 
            + DiffusionTerm(coeff=z1*mig*phi2,var=deltaC1)
            + DiffusionTerm(coeff=z1*mig*c1,var=deltaPhi2)
            # subtract Faradaic term
            - bv*deltaC1/c1_0*numerix.exp(z1*beta*(phi_1-phi2-phi_0))
            + bv*deltaC2/c2_0   
            - bv*c1/c1_0*numerix.exp(z1*beta*(phi_1-phi2-phi_0))*(-beta*deltaPhi2) 
            # incorporate Robin BC at x =0
            + PowerLawConvectionTerm(coeff=c1_a)
            + DiffusionTerm(coeff=b) 
            + ResidualTerm(equation=eq1)   )

newton_eqC2 = (TransientTerm(var=deltaC1)
            == DiffusionTerm(coeff=D*c2.grad,var=deltaC2) 
            # subtract Faradaic term
            + bv*deltaC1/c1_0*numerix.exp(z1*beta*(phi_1-phi2-phi_0))
            - bv*deltaC2/c2_0   
            + bv*c1/c1_0*numerix.exp(z1*beta*(phi_1-phi2-phi_0))*(-beta*deltaPhi2)
            + ResidualTerm(equation=eq3) )

newton_eqPhi2 = ( DiffusionTerm(coeff=z1*D*c1.grad,var=deltaC1) 
            + DiffusionTerm(coeff=mig*phi2,var=deltaC1)
            + DiffusionTerm(coeff=mig*c1,var=deltaPhi2)
            # add Faradaic term
            + bv*deltaC1/c1_0*numerix.exp(z1*beta*(phi_1-phi2-phi_0))
            - bv*deltaC2/c2_0   
            + bv*c1/c1_0*numerix.exp(z1*beta*(phi_1-phi2-phi_0))*(-beta*deltaPhi2)  
            # Add Robin BCs
            + PowerLawConvectionTerm(coeff=phi_a)
            + DiffusionTerm(coeff=phi_b) + (g*mask*mesh.faceNormals).divergence 
            + ResidualTerm(equation=eq3)    )

# deltaC1.constrain(0.,where=mesh.facesLeft) ... don't understand this
# deltaC2.constrain(...)
# deltaPhi2.constrain(...)

# Set values for c1,c2,phi2, and their deltas to 0.?.

newtonC1 = []
newtonC2 = []
newtonPhi2 = []

c1.updateOld()
c2.updateOld()
phi2.updateOld()
deltaC1.updateOld()
deltaC2.updateOld()
deltaPhi2.updateOld()
for sweep in range(20):
    res_c1 = newton_eqC1.sweep(dt=1)
    res_c2 = newton_eqC2.sweep(dt=1)
    res_phi2 = newton_eqPhi2.sweep(dt=1)
    newtonC1.append([sweep,res_c1,max(abs(deltaC1))])
    newtonC2.append([sweep,res_c2,max(abs(deltaC2))])
    newtonPhi2.append([sweep,res_phi2,max(abs(deltaPhi2))])